In [13]:
import torch 
import torch.nn as nn
import torchtext.vocab as vocab

In [14]:
vocab.pretrained_aliases.keys()
[key for key in vocab.pretrained_aliases.keys() if 'glove'in key]

['glove.42B.300d',
 'glove.840B.300d',
 'glove.twitter.27B.25d',
 'glove.twitter.27B.50d',
 'glove.twitter.27B.100d',
 'glove.twitter.27B.200d',
 'glove.6B.50d',
 'glove.6B.100d',
 'glove.6B.200d',
 'glove.6B.300d']

In [15]:
cache_dir = '/home/data'
glove = vocab.GloVe(name='6B',dim=50,cache=cache_dir)

In [16]:
print('一共包含%d个词'%(len(glove.stoi)))
glove.stoi['china'],glove.itos[3366]
glove.vectors[3366]

一共包含400000个词


tensor([ 0.5462,  1.2042, -1.1288, -0.1325,  0.9553,  0.0405, -0.4786, -0.3397,
        -0.2806,  0.7176, -0.5369, -0.0046,  0.7322,  0.1210,  0.2809, -0.0881,
         0.5973,  0.5526,  0.0566, -0.5025, -0.6320,  1.1439, -0.3105,  0.1263,
         1.3155, -0.5244, -1.5041,  1.1580,  0.6880, -0.8505,  2.3236, -0.4179,
         0.4452, -0.0192,  0.2897,  0.5326, -0.0230,  0.5896, -0.7240, -0.8522,
        -0.1776,  0.1443,  0.4066, -0.5200,  0.0908,  0.0830, -0.0220, -1.6214,
         0.3458, -0.0109])

In [17]:
def knn(W,x,k):
    cos = torch.matmul(W,x.view((-1,)))/torch.sqrt(torch.sum(W*W,dim=1)*torch.sum(x*x)+1e-9)
    _,topk = torch.topk(cos,k=k)
    topk = topk.cpu().numpy()
    return topk,[cos[i].item()for i in topk]

In [18]:
def get_similar_tokens(query_token,k,embed):
    topk,cos = knn(embed.vectors,embed.vectors[embed.stoi[query_token]],k=k+1)
    for i,c in zip(topk[1:],cos[1:]):
        print('cosine sim=%.3f: %s'%(c,embed.itos[i]))


In [19]:
get_similar_tokens('chip',3,glove)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [20]:
get_similar_tokens('baby',3,glove)


cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [21]:
get_similar_tokens('beautiful',3,glove)


cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


In [22]:
def get_analogy(token_a,token_b,token_c,embed):
    vecs = [embed.vectors[embed.stoi[t]] for t in [token_a,token_b,token_c]]
    x = vecs[2]+vecs[1]-vecs[0]
    topk,cos = knn(embed.vectors,x,1)
    return embed.itos[topk[0]]


In [24]:
get_analogy('man','women','son',glove)

'daughters'

In [25]:
get_analogy('beijing','china','tokyo',glove)

'japan'